In [1]:
import polars as pl
import pandas as pd 
import numpy as np 
import plotly.graph_objects as go 
import gc 

In [ ]:
df1_pl = pl.from_pandas(pd.read_excel("input/20250715-ketquathi-ct2018a.xlsx", sheet_name="Sheet1").astype(str))
df2_pl = pl.from_pandas(pd.read_excel("input/20250715-ketquathi-ct2018a.xlsx", sheet_name="Sheet2").astype(str))

df = pl.concat([df1_pl, df2_pl], how="vertical").lazy()
df = df.with_columns([
    pl.col("Toán").str.replace("nan", "0").alias("Toán"),
    pl.col("Văn").str.replace("nan", "0").alias("Văn"),
    *[
        pl.when(pl.col(col) == "nan").then(None).otherwise(pl.col(col)).alias(col)
        for col in df.collect_schema().names()
        if col not in ["Toán", "Văn"]
    ]
])

del df1_pl, df2_pl
gc.collect()
print(df.head().collect())


In [ ]:
df.collect().height

In [ ]:
NGOAI_NGU = {
    "N1" : "Anh", 
    "N2" : "Nga", 
    "N3" : "Pháp", 
    "N4" : "Trung",
    "N5" : "Đức", 
    "N6" : "Nhật", 
    "N7" : "Hàn"
}

ngoai_ngu_table = (
    df
    .select(["SOBAODANH", "Mã môn ngoại ngữ", "Ngoại ngữ"])
    .filter(
        (pl.col("Ngoại ngữ").is_not_null()) | (pl.col("Ngoại ngữ") != "nan") |
        (pl.col("Mã môn ngoại ngữ").is_not_null()) | (pl.col("Ngoại ngữ") != "nan")
    )
    .with_columns(
        pl.col("Mã môn ngoại ngữ").replace(NGOAI_NGU).alias("Mã môn ngoại ngữ")
    )
    .collect()
    .pivot(
        on = "Mã môn ngoại ngữ",
        index = "SOBAODANH", 
        values = "Ngoại ngữ",
    )
).lazy()

ngoai_ngu_table.collect().head()

In [ ]:
result_df = df.drop(["Ngoại ngữ", "Mã môn ngoại ngữ"]).join(ngoai_ngu_table,on="SOBAODANH",how="full").drop("SOBAODANH_right").with_columns(
    pl.col("SOBAODANH")
        .str.zfill(2)
        .str.slice(0, 2)
        .alias("Mã tỉnh thành")
)
result_df.collect().head()

In [ ]:
result_df.collect().write_csv("output/20250715-ketquathi-ct2018a.csv")

In [ ]:
result_df.collect().write_parquet("output/20250715-ketquathi-ct2018a.parquet")